Electric Vehicles Market Size Analysis uygulamamız için modelimizi kaydedeceğiz

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
# Veriyi Yüklüyoruz

In [4]:
df = pd.read_csv("Electric_Vehicle_Population_Data.csv")

In [5]:
df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,5YJYGDEE1L,King,Seattle,WA,98122.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,37.0,125701579,POINT (-122.30839 47.610365),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
1,7SAYGDEE9P,Snohomish,Bothell,WA,98021.0,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,1.0,244285107,POINT (-122.179458 47.802589),PUGET SOUND ENERGY INC,5.306105e+10
2,5YJSA1E4XK,King,Seattle,WA,98109.0,2019,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,270,0,36.0,156773144,POINT (-122.34848 47.632405),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
3,5YJSA1E27G,King,Issaquah,WA,98027.0,2016,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,5.0,165103011,POINT (-122.03646 47.534065),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
4,5YJYGDEE5M,Kitsap,Suquamish,WA,98392.0,2021,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,23.0,205138552,POINT (-122.55717 47.733415),PUGET SOUND ENERGY INC,5.303594e+10


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177866 entries, 0 to 177865
Data columns (total 17 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   VIN (1-10)                                         177866 non-null  object 
 1   County                                             177861 non-null  object 
 2   City                                               177861 non-null  object 
 3   State                                              177866 non-null  object 
 4   Postal Code                                        177861 non-null  float64
 5   Model Year                                         177866 non-null  int64  
 6   Make                                               177866 non-null  object 
 7   Model                                              177866 non-null  object 
 8   Electric Vehicle Type                              177866 non-null  object

In [7]:
df.isnull().sum()

VIN (1-10)                                             0
County                                                 5
City                                                   5
State                                                  0
Postal Code                                            5
Model Year                                             0
Make                                                   0
Model                                                  0
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         0
Base MSRP                                              0
Legislative District                                 389
DOL Vehicle ID                                         0
Vehicle Location                                       9
Electric Utility                                       5
2020 Census Tract                                      5
dtype: int64

In [8]:
# Veri Ön İşleme

In [9]:
data_clean = df[df['Electric Range'] > 0].copy()

In [10]:
# Kullanılacak Özellikler

In [11]:
X = data_clean[['Model Year', 'Make', 'Electric Vehicle Type']]

In [12]:
y = data_clean['Electric Range']

In [13]:
# Eğitim ve Test setlerine ayıracağız

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# 3. Pipeline Kurulumu (Kategorik verileri otomatik dönüştürmek için)

In [17]:
numeric_features = ['Model Year']

In [18]:
categorical_features = ['Make', 'Electric Vehicle Type']

In [19]:
from sklearn.compose import ColumnTransformer

In [20]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [21]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [22]:
# Model Tanımlama (Random Forest)

In [23]:
from sklearn.pipeline import Pipeline

In [24]:
from sklearn.ensemble import RandomForestRegressor

In [25]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

In [26]:
# Modeli Eğitme

In [27]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Model Year']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Make',
                                                   'Electric Vehicle Type'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

In [28]:
pipeline.score(X_train, y_train)

0.986434724366504

In [29]:
# Değerlendirme

In [30]:
y_pred = pipeline.predict(X_test)

In [31]:
from sklearn.metrics import mean_squared_error, r2_score

In [32]:
mse = mean_squared_error(y_test, y_pred)

In [33]:
print(f"Mean Squared Error: {mse:.2f}")

Mean Squared Error: 141.33


In [34]:
r2 = r2_score(y_test, y_pred)

In [35]:
print(f"R2 Score: {r2:.4f}")

R2 Score: 0.9857


In [36]:
# Model save

In [37]:
import joblib

In [38]:
joblib.dump(pipeline, 'ev_range_model.pkl');

pipeline.score'umuz 0.99 R2 Score: 0.9857